<a href="https://colab.research.google.com/github/rahiakela/machine-learning-algorithms/blob/main/neural-networks-from-scratch/04-activation-function/activation_functions_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ReLU Activation Function from Scratch

The rectified linear activation function is simpler than the sigmoid. It’s quite literally $y=x$ , clipped at $\theta$ from the negative side. If $x$ is less than or equal to $\theta$ , then $y$ is $\theta$ — otherwise, $y$ is equal to $x$.

$$
y = {\displaystyle \textstyle {\begin{cases} x, \space \space  x > 0 \\ 0, \space \space x < 0 \end{cases}}}
$$

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/neural-networks-from-scratch/04-activation-function/images/1.png?raw=1' width='600'/>

This simple yet powerful activation function is the most widely used activation function at the time of writing for various reasons — mainly speed and efficiency.

The ReLU activation function is extremely close to being a linear activation
function while remaining nonlinear, due to that bend after 0. This simple property is, however, very effective.




##Setup

In [ ]:
!pip install nnfs

In [2]:
from nnfs.datasets import spiral_data
import numpy as np
import nnfs
import matplotlib.pyplot as plt

nnfs.init()

## ReLU Activation 

Despite the fancy sounding name, the rectified linear activation function is straightforward to code. Most closely to its definition:

In [3]:
inputs = [0, 2, -1, 3.3, -2.7, 1.1, 2.2, -100]

output = []

for i in inputs:
  if i > 0:     # if the current value is greater than 0, appending the current value
    output.append(i)
  else:         # if it’s not, appending 0
    output.append(0)

print(output)

[0, 2, 0, 3.3, 0, 1.1, 2.2, 0]


This can be written more simply, as we just need to take the largest of two values: 0 or neuron value. 

For example:

In [4]:
inputs = [0, 2, -1, 3.3, -2.7, 1.1, 2.2, -100]

output = []

for i in inputs:
    output.append(max(0, i))

print(output)

[0, 2, 0, 3.3, 0, 1.1, 2.2, 0]


NumPy contains an equivalent — `np.maximum()`:

In [5]:
inputs = [0, 2, -1, 3.3, -2.7, 1.1, 2.2, -100]

output = np.maximum(0, inputs)

print(output)

[0.  2.  0.  3.3 0.  1.1 2.2 0. ]


This method compares each element of the input list (or an array) and returns an object of the same shape filled with new values. 

We will use it in our new rectified linear activation class:

In [6]:
# ReLU activation class
class ReLU:
  # Forward pass
  def forward(self, inputs):
    # Calculate output values from input
    self.output = np.maximum(0, inputs)

    return self.output

In [7]:
relu = ReLU()
print(relu.forward(inputs))

[0.  2.  0.  3.3 0.  1.1 2.2 0. ]


Let’s apply this activation function to the dense layer’s outputs.

In [8]:
class Dense:

  def __init__(self, n_inputs, n_neurons):
    """Layer initialization: Initialize weights and biases"""
    # Note that we’re initializing weights to be (inputs, neurons), rather than ( neurons, inputs)
    self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
    # a bias can ensure that a neuron fires initially. so initializing it with zero
    self.biases = np.zeros((1, n_neurons))

  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = np.dot(inputs, self.weights) + self.biases

# ReLU activation class
class ReLU:
  # Forward pass
  def forward(self, inputs):
    # Calculate output values from input
    self.output = np.maximum(0, inputs)

In [9]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 3 output values
dense1 = Dense(2, 3)

# Create ReLU activation (to be used with Dense layer)
relu = ReLU()

# Make a forward pass of our training data through this layer
dense1.forward(X)

# Forward pass through activation func.
# Takes in output from previous layer
relu.forward(dense1.output)

# Let's see output of the first few samples
print(f"Before ReLU:\n {dense1.output[:5]}")
print(f"After ReLU:\n {relu.output[:5]}")

Before ReLU:
 [[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.0475188e-04  1.1395361e-04 -4.7983500e-05]
 [-2.7414842e-04  3.1729150e-04 -8.6921798e-05]
 [-4.2188365e-04  5.2666257e-04 -5.5912682e-05]
 [-5.7707680e-04  7.1401405e-04 -8.9430439e-05]]
After ReLU:
 [[0.         0.         0.        ]
 [0.         0.00011395 0.        ]
 [0.         0.00031729 0.        ]
 [0.         0.00052666 0.        ]
 [0.         0.00071401 0.        ]]


As you can see, negative values have been clipped (modified to be zero). That’s all there is to the rectified linear activation function used in the hidden layer. 

##Softmax Activation

In our case, we’re looking to get this model to be a classifier, so we want an activation function meant for classification.

In this case, the rectified linear unit is unbounded, not normalized with other units, and exclusive. 
- **Not normalized** implies the values can be anything, an output of [12, 99, 318] is without context
- **exclusive** means each output is independent of the others

To address this lack of context, the softmax activation on the output data can take in non-normalized, or uncalibrated, inputs and produce a normalized distribution of probabilities for our classes.

In the case of classification, what we want to see is a prediction of which class the network “thinks” the input represents. 

This distribution returned by the softmax activation function represents confidence scores for each class and will add up to 1.

For example, if our network has a confidence distribution for two classes: 
`[0.45, 0.55]`, the prediction is the 2nd class, but the confidence in this
prediction isn’t very high. Maybe our program would not act in this case since it’s not very confident.

Here’s the function for the Softmax :

$$
S_{i,j} = \frac{e^{z_{i,j}}}{\sum_{l=1}^L e^{z_{i,j}}}
$$

The first step for us is to “exponentiate” the outputs. We do this with Euler’s number, $e$, which is roughly `2.71828182846` and referred to as the “exponential growth” number.

Both the numerator and the denominator of the Softmax function contain $e$ raised to the power of $z$ , where $z$ , given indices, means a singular output value — the index $i$ means the current sample and the index $j$ means the current output in this sample. 

The numerator exponentiates the current output value and the denominator takes a sum of all of the exponentiated outputs for a given sample.

In [10]:
# Values from the previous output when we described what a neural network is
layer_outputs = [4.8, 1.21, 2.385]

# e - mathematical constant, we use E here to match a common coding style where constants are uppercased
E = 2.71828182846  # you can also use math.e

# For each value in a vector, calculate the exponential value
exp_values = []
for output in layer_outputs:
  exp_values.append(E ** output)  # ** - power operator in Python

print("exponentiated values:")
print(exp_values)

exponentiated values:
[121.51041751893969, 3.3534846525504487, 10.85906266492961]


To calculate the probabilities, we need non-negative values. Imagine the output as `[ 4.8 , 1.21 , - 2.385 ]` — even after normalization, the last
value will still be negative since we’ll just divide all of them by their sum.

A negative probability (or confidence) does not make much sense. An exponential value of any number is always non-negative — it returns 0 for negative infinity, 1 for the input of 0, and increases for positive values:

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/neural-networks-from-scratch/04-activation-function/images/2.png?raw=1' width='600'/>

The exponential function is a monotonic function. This means that, with higher input values, outputs are also higher, so we won’t change the predicted class after applying it while making sure that we get non-negative values. It also adds stability to the result as the normalized exponentiation is more about the difference between numbers than their magnitudes.

Once we’ve exponentiated, we want to convert these numbers to a probability distribution (converting the values into the vector of confidences, one for each class, which add up to 1 for everything in the vector). What that means is that we’re about to perform a normalization where we take a given
value and divide it by the sum of all of the values.

Since each output value normalizes to a fraction of the sum, all of the values are now in the range of 0 to 1 and add up to 1 — they share the probability of 1 between themselves.

Let’s add the sum and normalization to the Softmax.

In [11]:
# Values from the previous output when we described what a neural network is
layer_outputs = [4.8, 1.21, 2.385]

# e - mathematical constant, we use E here to match a common coding style where constants are uppercased
E = 2.71828182846  # you can also use math.e

# For each value in a vector, calculate the exponential value
exp_values = []
for output in layer_outputs:
  exp_values.append(E ** output)  # ** - power operator in Python

print("exponentiated values:")
print(exp_values)

# Now normalize values
norm_base = sum(exp_values)  # we sum all values
norm_values = []
for value in exp_values:
  norm_values.append(value / norm_base)

print("Normalized exponentiated values:")
print(norm_values)
print(f"Sum of normalized values:{sum(norm_values)}")

exponentiated values:
[121.51041751893969, 3.3534846525504487, 10.85906266492961]
Normalized exponentiated values:
[0.8952826639573506, 0.024708306782070668, 0.08000902926057876]
Sum of normalized values:1.0


We can perform the same set of operations with the use of NumPy in the following way:

In [12]:
# Values from the previous output when we described what a neural network is
layer_outputs = [4.8, 1.21, 2.385]

# For each value in a vector, calculate the exponential value
exp_values = np.exp(layer_outputs)
print("exponentiated values:")
print(exp_values)

# Now normalize values
norm_base = exp_values / np.sum(exp_values)  # we sum all values
print("Normalized exponentiated values:")
print(norm_values)
print(f"Sum of normalized values:{np.sum(norm_values)}")

exponentiated values:
[121.51041752   3.35348465  10.85906266]
Normalized exponentiated values:
[0.8952826639573506, 0.024708306782070668, 0.08000902926057876]
Sum of normalized values:1.0


Notice the results are similar, but faster to calculate and the code is easier to read with NumPy.

We can exponentiate all of the values with a single call of the `np.exp()`, then immediately normalize them with the sum. To train in batches, we need to convert this functionality to accept layer outputs in batches.